# Molecules, Graphs, and MoleculeSets

At the heart of PyProteoNet are types of molecules (like proteins and peptides) and connections between those molecules. Even though for PyProteoNet molecules
are just nodes in a graph and molecule types are just strings we will focus on protein and peptide molecules as used
during many mass spectrometry (MS) experiments in the field of proteomics.
In addition, most functions of PyProteonNet are currenlty focused on proteins and peptides aggregation and imputation.

Ultimately most experiments want to measure protein abundances. However for most MS experiments proteins are digested into peptides
because only those smaller peptides are measured during an MS-experiment. So we have two kind of molecules (proteins and peptides) plus a mapping between
them because every peptide can be mapped to at least one protein which the peptide can result from during digestion.

In PyProteoNet such a group of different types of molecules together with mappings between those molecule types is represented by a `MoleculeSet`.
So lets import this:

In [1]:
from pyproteonet.data import MoleculeSet

Next we need some data to create a MoleculeSet. For simplicity, we do not use real data but come up with a simple toy examples
of 10 proteins and 100 peptides which we identify by integers.

In [2]:
import pandas as pd

proteins = pd.DataFrame(index=range(10))
peptides = pd.DataFrame(index=range(100))

As you can see proteins and peptides are represented by pandas dataframes. The index of the dataframes will be used as identifiers
for our molecules. With real-world data it might make sense to use the protein name or peptide sequence as index but here we just
use integers. Our dataframe could also have additional columns storing other molecule attributes, however, those are not required.

The only thing missing is a mapping between proteins and peptides.
Mappings are also created from pandas dataframes. To identify mapping partners those dataframes must have a multiindex with every index level containing molecules ids of the mapped molecules.

Here we just map every 10th peptide to the same protein.

In [3]:
peptide_protein_mapping = pd.DataFrame({'peptide':peptides.index, 'protein':peptides.index%10}).set_index(['peptide', 'protein'])

In [4]:
peptide_protein_mapping

,
peptide,protein
0,0
1,1
2,2
3,3
4,4
...,...
95,5
96,6
97,7


> **Side Note**: Internally mappings are wrapped by the `MoleculeMapping` class. This wrapping class can hold some additional information and facilitates e.g. mappings between the same molecule type. However, for most simple use cases like the protein-peptide use case this can be ignored.

From this data we can now generate a MoleculeSet. To support arbitratry molecule types and multiple mappings, molecules and
mappings need to be given as dictionaries.

In [5]:
ms = MoleculeSet(molecules = {'protein':proteins, 'peptide':peptides},
                 mappings = {'peptide-protein': peptide_protein_mapping}
                )

# Samples and Values

The MoleculeSet alone is not that helpful. Usually, we also want to attach (abundance) values to our molecules. 
With MS experiments we usually even have multiple samples measuring the same value multiple times. For this PyProteoNet provides
`Dataset`s. A `Dataset` consists of a molecule graph and a variable number of `DatasetSample`s each representing values of one sample.
We can create a `Dataset` without any samples as follows:

In [6]:
from pyproteonet.data import Dataset
ds = Dataset(molecule_set=ms)

Next we add some samples to the dataset. Every sample is identified by a name and contains dataframes with values for our
different molecule types. E.g. lets assume we measured some abundance values for our peptides which we want to add.

In [7]:
import numpy as np
for i in range(3):
    sample_name = f'sample{i}'
    peptide_values = pd.DataFrame({'abundance': np.random.uniform(size=100) * 10000}, index=range(100))
    sample_molecule_values = {'peptide':peptide_values}
    ds.create_sample(name=sample_name, values=sample_molecule_values)

Again we give the sample values as a dictionary to assign them to the correct molecule type.

# Protein Aggregation

Since in proteomics it is often worked with logarithmic abundance values, also the peptide abundances of our artifical dataset should first be logarithmized.

This also helps to understand how to access and modify dataset values in PyProteoNet. One convinient way shown here is to access a single value field or column as padas DataFrame in long format, containing all abundance values with their sample and protein id as multi index. Logarithmization can then be done as shown below, saving the logarithmized values under a new column.

In [8]:
ds.values['peptide']['abundance_log'] = np.log(ds.values['peptide']['abundance'])

Usually we are interested in protein abundances. Therefore, the measured peptide abundance values need to be aggregated into protein abundance values.
This is done via the peptide-protein mapping using an aggregation function (also called quantification function).

Here we apply Top3 aggregation as a simple but commonly used aggregation function. This function computes protein abundance from the average of the three most abundant peptides corresponding to a protein. In real-world datasets some peptides are usually shared between different proteins. Since their abundance values cannot be uniquly assigned to a protein, shared peptides are often ignored during abundane aggregation and only unique peptides are considered.

The result is represented as a pandas Series in long format with a multiindex to identify samples and protein ids.

In [9]:
from pyproteonet.quantification.neighbor_summarization import neighbor_top_n_mean
top3 = neighbor_top_n_mean(dataset=ds, molecule='protein', mapping='peptide-protein', partner_column='abundance_log',
                           top_n=3, only_unique=True)

In [10]:
top3

sample   id
sample0  0     8.872365
         1     8.939153
         2     8.863029
         3     9.060880
         4     9.042643
         5     9.026096
         6     8.802985
         7     8.881347
         8     9.119931
         9     9.114495
sample1  0     8.902748
         1     9.074433
         2     9.064276
         3     9.099389
         4     8.822320
         5     9.099863
         6     9.041178
         7     8.861726
         8     9.070201
         9     9.083218
sample2  0     8.861611
         1     8.974664
         2     9.096262
         3     9.082090
         4     9.149069
         5     9.058339
         6     9.126747
         7     8.843175
         8     8.942209
         9     9.157035
Name: quanti, dtype: float64

To assign this to our dataset the following syntax can be used:

In [11]:
ds.values['protein']['top3'] = top3

Alternatively, most functions also allow the direct specification of a result column. So an alternative formulation of the Top3 aggregation could be as follows:

In [12]:
_ = neighbor_top_n_mean(dataset=ds, molecule='protein', mapping='peptide-protein', partner_column='abundance_log',
                        top_n=3, only_unique=True, result_column='top3')

The long format of the Top3 aggregated protein abundance values is little intuitive and a matrix representation is often used instead. Therefore, the `Dataset` class provides functions to represent data in different formats allowing. To get the Top3 results as a pandas DataFrame with samples as columns ans proteins as rows the `get_samples_value_matrix` function can be useful:

In [13]:
ds.get_samples_value_matrix(molecule='protein', column='top3')

,sample0,sample1,sample2
id,,,
0,8.872365,8.902748,8.861611
1,8.939153,9.074433,8.974664
2,8.863029,9.064276,9.096262
3,9.060880,9.099389,9.082090
4,9.042643,8.822320,9.149069
5,9.026096,9.099863,9.058339
6,8.802985,9.041178,9.126747
7,8.881347,8.861726,8.843175
8,9.119931,9.070201,8.942209


Next to the rather simple neighbor average (Top3), PyProteoNet also provides an efficient implementation of the more complex MaxLFQ protein aggregation method propose by [Cox et al.](https://www.mcponline.org/article/S1535-9476(20)33310-7/fulltext).

This methods takes peptide abundance ratios between all samples into account and then solves a least squares optimization probel to find protein abundance values best representing the observed peptide abundances. 
Similar to [Cox et al.](https://www.mcponline.org/article/S1535-9476(20)33310-7/fulltext) we here require at least two non missing peptide abundances. Additionally, we need to specify that the given values are already logarithmic to allows the correct calculation of peptide ratios between samples.

In [21]:
from pyproteonet.quantification.maxlfq import maxlfq
_ = maxlfq(dataset=ds, molecule='protein', mapping='peptide-protein', partner_column='abundance_log',
           min_ratios=2, median_fallback=False, result_column='maxlfq', is_log=True)

Next to the matrix representation we can also look at all columns of a molecule using its long format representation as pandas DataFrame with multiindex. To do so we can again use the `values` attribute of our dataset. Using the `df` shortcut, we get a DataFrame of all columns for a molecule type in long format.

In [15]:
ds.values['protein'].df

top3    maxlfq
sample  id                    
sample0 0   8.872365  7.916048
        1   8.939153  8.116546
        2   8.863029  7.951180
        3   9.060880  8.385782
        4   9.042643  8.517170
        5   9.026096  7.632889
        6   8.802985  8.231608
        7   8.881347  8.254537
        8   9.119931  8.494972
        9   9.114495  8.309979
sample1 0   8.902748  7.974116
        1   9.074433  8.515271
        2   9.064276  7.943332
        3   9.099389  8.237472
        4   8.822320  8.253146
        5   9.099863  8.301372
        6   9.041178  8.410506
        7   8.861726  8.241358
        8   9.070201  8.225695
        9   9.083218  8.224337
sample2 0   8.861611  7.925707
        1   8.974664  8.355557
        2   9.096262  8.354092
        3   9.082090  8.143474
        4   9.149069  8.061111
        5   9.058339  8.159243
        6   9.126747  8.569846
        7   8.843175  8.081531
        8   8.942209  8.372671
        9   9.157035  8.141000

# Missing Value Imputation

Pyproteonet provides a wide range of established as well as newly proposed, graph neural network (GNN) based missing value imputation functions.

The interface for most imputation functions is similar to this of the aggregation functions shown above. Next to a dataset you need to provide the molecole type as well as the column(s) to impute and, optionally, values for method specific hyperparameters.

Of course, for imputation, we first of all need some missing values. So for our example we just mask some of the Top3 values. To do so we, again, use the `values` attribute of our dataset to get all Top3 values as pandas DataFrame in long format. Then, we replace some of the values with `Na` using pandas and, finally, we write the result back as a new column in our dataset

In [16]:
vals = ds.values['protein']['top3']
vals.loc[vals.sample(frac=0.33).index] = np.nan
ds.values['protein']['top3_masked'] = vals

In [17]:
ds.values['protein'].df

top3    maxlfq  top3_masked
sample  id                                 
sample0 0   8.872365  7.916048     8.872365
        1   8.939153  8.116546          NaN
        2   8.863029  7.951180     8.863029
        3   9.060880  8.385782          NaN
        4   9.042643  8.517170     9.042643
        5   9.026096  7.632889     9.026096
        6   8.802985  8.231608          NaN
        7   8.881347  8.254537     8.881347
        8   9.119931  8.494972          NaN
        9   9.114495  8.309979     9.114495
sample1 0   8.902748  7.974116     8.902748
        1   9.074433  8.515271          NaN
        2   9.064276  7.943332          NaN
        3   9.099389  8.237472     9.099389
        4   8.822320  8.253146     8.822320
        5   9.099863  8.301372     9.099863
        6   9.041178  8.410506          NaN
        7   8.861726  8.241358     8.861726
        8   9.070201  8.225695     9.070201
        9   9.083218  8.224337     9.083218
sample2 0   8.861611  7.925707     8.861611
        1   8.974664  8.355557          NaN
        2   9.096262  8.354092     9.096262
        3   9.082090  8.143474     9.082090
        4   9.149069  8.061111          NaN
        5   9.058339  8.159243     9.058339
        6   9.126747  8.569846     9.126747
        7   8.843175  8.081531     8.843175
        8   8.942209  8.372671     8.942209
        9   9.157035  8.141000          NaN

Let's use the commonly used MissForrest imputation algorithm to impute the missing values

In [18]:
from pyproteonet.imputation.r.miss_forest import impute_miss_forest
ds.values['protein']['top3_imputed'] = impute_miss_forest(dataset=ds, molecule='protein', column='top3_masked')

Looking at the result we can see that the missing values are gone:

In [19]:
ds.values['protein'].df

top3    maxlfq  top3_masked  top3_imputed
sample  id                                               
sample0 0   8.872365  7.916048     8.872365      8.872365
        1   8.939153  8.116546          NaN      8.986034
        2   8.863029  7.951180     8.863029      8.863029
        3   9.060880  8.385782          NaN      9.088491
        4   9.042643  8.517170     9.042643      9.042643
        5   9.026096  7.632889     9.026096      9.026096
        6   8.802985  8.231608          NaN      8.986034
        7   8.881347  8.254537     8.881347      8.881347
        8   9.119931  8.494972          NaN      8.999805
        9   9.114495  8.309979     9.114495      9.114495
sample1 0   8.902748  7.974116     8.902748      8.902748
        1   9.074433  8.515271          NaN      8.993970
        2   9.064276  7.943332          NaN      9.012298
        3   9.099389  8.237472     9.099389      9.099389
        4   8.822320  8.253146     8.822320      8.822320
        5   9.099863  8.301372     9.099863      9.099863
        6   9.041178  8.410506          NaN      8.993970
        7   8.861726  8.241358     8.861726      8.861726
        8   9.070201  8.225695     9.070201      9.070201
        9   9.083218  8.224337     9.083218      9.083218
sample2 0   8.861611  7.925707     8.861611      8.861611
        1   8.974664  8.355557          NaN      9.005234
        2   9.096262  8.354092     9.096262      9.096262
        3   9.082090  8.143474     9.082090      9.082090
        4   9.149069  8.061111          NaN      8.934685
        5   9.058339  8.159243     9.058339      9.058339
        6   9.126747  8.569846     9.126747      9.126747
        7   8.843175  8.081531     8.843175      8.843175
        8   8.942209  8.372671     8.942209      8.942209
        9   9.157035  8.141000          NaN      9.101985

If you look at the import of the impute_miss_forest function you will notice that this function is part of the "r" subpackage.
Most established imputation algorithms are implemented in the R programming language and provided as R packages. To also provide those algorithms while maintaining a unified Python interfact PyProteoNet wraps those R packages. 
Therefore, all algorithms in the "r" subpackage require an existing R installation (if you use a conda/mamba environment you could simply install one with the command `conda install -c conda-forge r-base`). For the user it is transparent whether an imputation function is implemented in Python or wrapped from an R package. All installation of R dependencies and conversion of data types between Python and R is done in the background by PyProteoNet (internally the rpy2 package is used for this).

> **Note**: Compared to other proteomics- or imputation-focused packages PyProteoNet allows to jointly manage protein and peptide values as well as the relation between them (plus any other additional molecule types if required). This together with the implemented aggragation and imputation functions allows for a more versatile usage scenarios. E.g. imputation can be applied both on peptide level (before aggregation) as well as on protein level (after aggregation). In addition, the application and comparision of different imputation algorithmis and stratigies on the same dataset is facilitated

## Graph Neural Network Imputation

While traditionally, imputation is either applied on peptide OR on protein level modelling protein and peptides a graph structure allows for flexilbe imputation strategies jointly taking information from both molecules into account. Therefore, imputation is formulated as a regression problem on the protein-peptide graph which is then solved by training a graph neural network (GNN).

While PyProteoNet provides different flavors and implementations using different network architectures and training schemes for the underlying GNN, those imputation methods can be called via a similar interface as other imputation methods.
Since two types of molecules (proteins and peptides) are taken into account, the name of those molecule types as well as two value columns have to be specified.

Additional hyperparameters can be set aswell (here set to values used for real-world datasets)

In [22]:
from pyproteonet.imputation.dnn import impute_all_sample_gnn
impute_all_sample_gnn(dataset=ds, protein_abundance_column=f'top3_masked', peptide_abundance_column=f'abundance_log', result_column=f'gnnimp',
                      protein_molecule='protein', peptide_molecule='peptide',
                      peptide_masking_fraction=0.4, training_fraction=0.4, missing_substitute_value=-3, max_epochs=2000,
                      early_stopping_patience=5)

/hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteonet/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteo ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name   | Type           | Params
------------------------------------------
0 | _model | UncertaintyGAT | 816   
------------------------------------------
816       Trainable params
0         Non-trainable params
816       Total params
0.003     Total estimated model params size (MB)


[3, 3, 3]
[4, 2, 2]
3


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteonet/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteonet/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
/hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteonet/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:121: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured in

Training: |                                               | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

step299: validation_pearson:1.0 || validation_r2:1.0 || validation_loss:1.1189312934875488 || validation_mse:1.1189312934875488 || validation_rmse:1.057795524597168 || validation_mae:1.043043613433838 || validation_uncertainty_pearson:1.0 || epoch:299 || 


/hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteonet/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteo ...
/hpi/fs00/home/tobias.pietz/mambaforge/envs/pyproteonet/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting: |                                             | 0/? [00:00<?, ?it/s]

sample   id
sample0  0     8.872365
sample1  0     8.902748
sample2  0     8.861611
sample0  1     8.997410
sample1  1     8.965415
sample2  1     8.997974
sample0  2     8.863029
sample1  2     8.994355
sample2  2     9.096262
sample0  3     8.990422
sample1  3     9.099389
sample2  3     9.082090
sample0  4     9.042643
sample1  4     8.822320
sample2  4     8.997708
sample0  5     9.026096
sample1  5     9.099863
sample2  5     9.058339
sample0  6     8.986112
sample1  6     9.005007
sample2  6     9.126747
sample0  7     8.881347
sample1  7     8.861726
sample2  7     8.843175
sample0  8     8.991920
sample1  8     9.070201
sample2  8     8.942209
sample0  9     9.114495
sample1  9     9.083218
sample2  9     8.995906
dtype: float64

In [23]:
ds.values['protein'].df

top3    maxlfq  top3_masked  top3_imputed    gnnimp
sample  id                                                         
sample0 0   8.872365  7.916048     8.872365      8.872365  8.872365
        1   8.939153  8.116546          NaN      8.986034  8.997410
        2   8.863029  7.951180     8.863029      8.863029  8.863029
        3   9.060880  8.385782          NaN      9.088491  8.990422
        4   9.042643  8.517170     9.042643      9.042643  9.042643
        5   9.026096  7.632889     9.026096      9.026096  9.026096
        6   8.802985  8.231608          NaN      8.986034  8.986112
        7   8.881347  8.254537     8.881347      8.881347  8.881347
        8   9.119931  8.494972          NaN      8.999805  8.991920
        9   9.114495  8.309979     9.114495      9.114495  9.114495
sample1 0   8.902748  7.974116     8.902748      8.902748  8.902748
        1   9.074433  8.515271          NaN      8.993970  8.965415
        2   9.064276  7.943332          NaN      9.012298  8.994355
        3   9.099389  8.237472     9.099389      9.099389  9.099389
        4   8.822320  8.253146     8.822320      8.822320  8.822320
        5   9.099863  8.301372     9.099863      9.099863  9.099863
        6   9.041178  8.410506          NaN      8.993970  9.005007
        7   8.861726  8.241358     8.861726      8.861726  8.861726
        8   9.070201  8.225695     9.070201      9.070201  9.070201
        9   9.083218  8.224337     9.083218      9.083218  9.083218
sample2 0   8.861611  7.925707     8.861611      8.861611  8.861611
        1   8.974664  8.355557          NaN      9.005234  8.997974
        2   9.096262  8.354092     9.096262      9.096262  9.096262
        3   9.082090  8.143474     9.082090      9.082090  9.082090
        4   9.149069  8.061111          NaN      8.934685  8.997708
        5   9.058339  8.159243     9.058339      9.058339  9.058339
        6   9.126747  8.569846     9.126747      9.126747  9.126747
        7   8.843175  8.081531     8.843175      8.843175  8.843175
        8   8.942209  8.372671     8.942209      8.942209  8.942209
        9   9.157035  8.141000          NaN      9.101985  8.995906